# Test Code and Snippets for Telegram API and Chat/Channel Scraping

Authentication and API development tool can be accessed at https://my.telegram.org/auth

# Telegram App Details
--- TODO: Remove this when making repo public ---`

- Short Name: TelegramTest
- Test Configuration: 149.154.167.40:443
- API ID: 23867482
- api_hash: 7c99e3fd9ecfd7b8a4a0374436a046a9


## Telethon
Let's take a look at Telethon/Asynchio for scraping

In [5]:
%%capture 
!pip install telethon
!pip install pandas

In [150]:
from telethon.sync import TelegramClient
import telethon.sync
import pandas as pd
import asyncio
import datetime
import pytz

In [40]:
# Credentials
api_id = 23867482
api_hash = '7c99e3fd9ecfd7b8a4a0374436a046a9'
phone = '+38631882318'
username = ""

### Code Snippets


In [ ]:
# Asynchio Hello World / Semantics
# Note: Asynchio syntax in jupyter >=7.0 has changed, you can now just call await from the top level in the notebook
async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())

await main()

#Snippet to Cancel current asyncio task
#curr_task = asyncio.current_task()
#curr_task.cancel()

In [48]:
# Telethon Hello World/Semantics
client = TelegramClient(None, api_id, api_hash)
# client.connect()
await client.start()
await client.send_message('me', 'Hello, world!')
await client.disconnect()

# To delete session
await client.log_out()

Please enter your phone (or bot token):  √sf
Please enter your phone (or bot token):  afaf
Please enter your phone (or bot token):  afsafa
Please enter your phone (or bot token):  asfasf
Please enter your phone (or bot token):  +38631882318


CancelledError: 

In [52]:
# Scrape Messages from Single Chat / Channel

# Chat ID

# This value corresponds to a test chat I made with a friend, should contain a message "Ignore this" as the first message in the chat
chat = -990435703

# This corresponds to a test channel I created, should contain a message "Ignore this" as the first message in the channel
chat = "t.me/TestChannelMtnDew"

client = TelegramClient(None, api_id, api_hash)
await client.start()
# ---- START ----

async def get_msgs():
    async for message in client.iter_messages(chat):
        print(message.sender_id, ':', message.text)
        
await get_msgs()

# ---- END ----
await client.log_out()

Please enter your phone (or bot token):  +38631882318
Please enter the code you received:  32150


Signed in successfully as Mtn Dew Camatcho
-1001558795535 : Ignore this
-1001558795535 : None


True

The eventual functionality needed is to be able to scrape all messages from chats and channels between two dates and 
store the message text and user ID (later we will also scrape URL's from the messages).

Some usefull parameters for `iter_messages`:
- limit (int | None, optional): Max no messages to retreive
- offset_date (datetime): Older messages than this will be retreived

Note that only one bound can be specified via the function parameters. A workaround is to fetch the oldest message wanted then fetch all
messages between it and the first message in range.


In [88]:
# Snippet to put date and time in required format
'''
from_date = datetime.strptime('27.03.2023 13:30:00', '%d.%m.%Y %H:%M:%S').replace(tzinfo=timezone.utc)
to_date = datetime.strptime('27.03.2023 14:15:00', '%d.%m.%Y %H:%M:%S').replace(tzinfo=timezone.utc)
'''

"\nfrom_date = datetime.strptime('27.03.2023 13:30:00', '%d.%m.%Y %H:%M:%S').replace(tzinfo=timezone.utc)\nto_date = datetime.strptime('27.03.2023 14:15:00', '%d.%m.%Y %H:%M:%S').replace(tzinfo=timezone.utc)\n"

In [179]:
# Helper function for getting the first and last message in a date ranges

async def sfetch_in_range(client, channel, from_date, to_date):
    # Get ID of oldest message in range via offset parameters
    
    pre_first_msg = await client.get_messages(channel, offset_date=from_date, limit=1)
    pre_first_msg = pre_first_msg[0]
    #first_msg = pre_first_msg
     
    # Cleaner, but breaks when there are is only a single message available
    # TODO: Fix this edge case

    first_msg = await client.get_messages(channel, min_id=pre_first_msg.id, limit=1, reverse=True)
    first_msg = first_msg[0]
    
    # Get ID of youngest message in range via offset parameters
    last_msg = await client.get_messages(channel, offset_date=to_date, limit=1)
    last_msg = last_msg[0]
    
    '''
    messages_between = await client.get_messages(channel, min_id=first_msg.id, max_id=last_msg.id)
    messages_between = messages_between + [first_msg, last_msg]
    '''
    
    return [first_msg, last_msg]

**Note:** When accessing a private channel you need to prefix the ID copied from the browser address with a 100
I.e. chat = -1280863188 becomes chat = -1001280863188

In [182]:
# Test helper function
# Expect to retrieve message "This message was sent at 14:00 pm", "And this one before 14:15" but not "Ignore this"

# This value corresponds to a test chat I made with a friend
#chat = -990435703

# Unkomercialer Marketplatz
chat = -1001280863188

from_date = datetime.datetime.strptime('27.03.2023 12:30:00', '%d.%m.%Y %H:%M:%S').replace(tzinfo=pytz.timezone('Europe/Zurich'))
to_date = datetime.datetime.strptime('27.03.2023 08:30:00', '%d.%m.%Y %H:%M:%S').replace(tzinfo=pytz.timezone('Europe/Zurich'))


client = TelegramClient(None, api_id, api_hash)
await client.start()


# ---- START ----

messages = await sfetch_in_range(client, chat, from_date, to_date)
#for message in messages:
#    print(message.text)
print(len(messages))


# ---- END ----
await client.log_out()


2


True

<DstTzInfo 'Europe/Zurich' LMT+0:34:00 STD>